0. 

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import datetime

In [2]:
data = pd.read_csv(r'D:\资料\大四上\社交网络挖掘\Foursquare\dataset_TSMC2014_TKY.csv',sep=',',header='infer')
userIDs = data["userId"].to_numpy()
userIDs = np.unique(userIDs)
venueIDs = data["venueId"].to_numpy()
venueIDs = np.unique(venueIDs)

In [155]:
print(data.columns)

Index(['userId', 'venueId', 'venueCategoryId', 'venueCategory', 'latitude',
       'longitude', 'timezoneOffset', 'utcTimestamp', 'year', 'month', 'day',
       'hour', 'minute', 'second', 'weekday'],
      dtype='object')


In [3]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

def get_time(utcTimestamp, timezoneOffset):
    
    time_data = utcTimestamp.split(' ')
    day, year, month, date, time = time_data[0], time_data[-1], time_data[1], time_data[2], time_data[3]
    hour, minute, second = time.split(':')
    dt = datetime.datetime(int(year), months.index(month)+1, int(date), int(hour), int(minute), int(second)) + datetime.timedelta(minutes = int(timezoneOffset))
    
    return dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, days.index(day)+1

In [4]:
data[["year", "month", "day", "hour", "minute", "second", "weekday"]] = data.apply(lambda x:get_time(x["utcTimestamp"], x["timezoneOffset"]), axis=1, result_type='expand')

In [5]:
save_path = "result/"
suffix = ".html"

地点类的分布

In [99]:
venueCategorys = data["venueCategory"].to_numpy()
venueCategorys, count=np.unique(venueCategorys, return_counts=True)
idx = np.argsort(-count)
venueCategorys = venueCategorys[idx]
count = count[idx]
print(venueCategorys[:20])

['Train Station' 'Subway' 'Ramen /  Noodle House' 'Convenience Store'
 'Japanese Restaurant' 'Bar' 'Food & Drink Shop' 'Electronics Store'
 'Mall' 'Coffee Shop' 'Office' 'Café' 'Bridge' 'Bus Station' 'Road'
 'Bookstore' 'Park' 'Fast Food Restaurant' 'Arcade'
 'College Academic Building']


In [13]:
fig = px.bar(pd.DataFrame({"venueCategory": venueCategorys[:20], "count": count[:20]}),
    x="venueCategory",
    y="count",
    title="venueCategory_count")
    
fig.write_html(save_path + "venueCategory_count" + suffix)

去掉 train station，干扰太大

In [44]:
clean_data = data[~data["venueCategory"].isin(["Train Station"])]
train_station_data = data[data["venueCategory"].isin(["Train Station"])]

月份的分布

In [62]:
fig = px.histogram(data, x="month")
fig.write_html(save_path + "month" + suffix)

不同类之间月份没什么区别

In [16]:
fig = px.histogram(data[data["venueCategory"].isin(venueCategorys[:50])], x="month", animation_frame="venueCategory", histnorm='percent')
fig.write_html(save_path + "month_venueCategory" + suffix)

weekday 的分布，周末和工作日有一点区别

In [17]:
fig = px.histogram(data, x="weekday")
fig.write_html(save_path + "weekday" + suffix)

不同类地点，确实有区别

In [18]:
fig = px.histogram(data[data["venueCategory"].isin(venueCategorys[:50])], x="weekday", animation_frame="venueCategory", histnorm='percent')
fig.write_html(save_path + "weekday_venueCategory" + suffix)

一天内时间的分布（hour）

In [19]:
fig = px.histogram(data, x="hour")
fig.write_html(save_path + "hour" + suffix)

In [77]:
venueCategory = venueCategorys[0]
month_data = data[data["venueCategory"] == venueCategory]["month"].to_numpy()
y = np.zeros(12)
for j in month_data:
    y[j-1] += 1

In [78]:
y

array([18155.,  9523.,     0., 27619., 32723., 22826., 22594.,  9598.,
        3636., 13179., 19393., 21182.])

In [90]:
fig = go.Figure()
x = np.zeros(12)
for i in range(10):
    venueCategory = venueCategorys[i]
    month_data = data[data["venueCategory"] == venueCategory]["month"].to_numpy()
    y = np.zeros(12) 
    for j in month_data:
        y[j-1] += 1
    y /= len(month_data)
    fig.add_trace(go.Scatter(		
    x=list(range(1, 13)),				
    y=y,
    name=venueCategory
))
fig.update_layout(
    margin=dict(l=20, r=30, t=20, b=20))
fig.show()

In [88]:
fig = go.Figure()
for i in range(10):
    venueCategory = venueCategorys[i]
    hour_data = data[data["venueCategory"] == venueCategory]["hour"].to_numpy()
    y = np.zeros(24) 
    for j in hour_data:
        y[j-1] += 1
    y /= len(hour_data)
    fig.add_trace(go.Scatter(		
    x=list(range(1, 25)),				
    y=y,
    name=venueCategory
))
fig.update_layout(
    margin=dict(l=20, r=30, t=20, b=20))
fig.show()

In [132]:
fig = go.Figure()
for i in range(10):
    venueCategory = venueCategorys[i]
    weekday_data = data[data["venueCategory"] == venueCategory]["weekday"].to_numpy()
    y = np.zeros(7) 
    for j in weekday_data:
        y[j-1] += 1
    y /= len(weekday_data)
    fig.add_trace(go.Scatter(		
    x=list(range(1, 8)),				
    y=y,
    name=venueCategory
))
fig.update_layout(
    margin=dict(l=20, r=30, t=20, b=20))
fig.show()

KeyError: 'weekday'

一天内时间也显然有区别

In [20]:
fig = px.histogram(data[data["venueCategory"].isin(venueCategorys[:50])], x="hour", animation_frame="venueCategory", histnorm='percent')
fig.write_html(save_path + "hour_venueCategory" + suffix)

单个/一类用户空间可视化

In [72]:
def draw_density(data, save_path, animation_frame=None, special_class=None, special_name=None, hover_data=None):
    if special_class is not None:
        new_data = data[data[special_class].isin(special_name)]
    else:
        new_data = data
    fig = px.density_mapbox(new_data, lat="latitude", lon="longitude", radius=1,
                        center=dict(lat=35.66, lon=139.73), zoom=10,
                        mapbox_style="open-street-map",
                        animation_frame=animation_frame,
                        hover_data=hover_data)
    fig.write_html(save_path)

单个用户

In [29]:
userID = np.random.choice(userIDs, 1)
print(userID)

[1972]


总体分布

In [38]:
draw_density(clean_data, save_path + "all_density" + suffix)

单个用户分布

In [54]:
draw_density(data, save_path + str(userID) + "_density" + suffix, special_class="userId", special_name=userID)

2. 总体分析在时间上的分布，再对单个/一类用户时间可视化（分为按日期和按一天内时段）

总体的 在一天内时间的分布

In [41]:
draw_density(clean_data, save_path + "all_hour_density" + suffix, "hour")

总体的 一周中的分布

In [45]:
draw_density(clean_data, save_path + "all_weekday_density" + suffix, "weekday")

地点之间地理位置的联系: 相邻地点是否更容易被用户访问

In [56]:
from geopy.distance import geodesic
dist = geodesic((37.193462, 100.1347), (37.194363, 100.1348)).m
print(dist)

100.3875225692039


In [58]:
from tqdm import tqdm

In [63]:
trajs = data.loc[data["userId"] == userID]
geodesic((trajs.iloc[0]['latitude'], trajs.iloc[0]['longitude']), (trajs.iloc[1]['latitude'], trajs.iloc[1]['longitude'])).m

8452.557549701727

In [65]:
distance = []
for userID in tqdm(userIDs):
    trajs = data.loc[data["userId"] == userID]
    for i in range(1, len(trajs)):
        distance.append(geodesic((trajs.iloc[i-1]['latitude'], trajs.iloc[i-1]['longitude']), (trajs.iloc[i]['latitude'], trajs.iloc[i]['longitude'])).m)

100%|██████████| 2293/2293 [09:27<00:00,  4.04it/s]


In [68]:
fig = px.histogram(pd.DataFrame({"distance": distance}),
    x="distance",
    nbins=100)
fig.write_html(save_path + "distance" + suffix)

有联系的用户是否意味着轨迹更相似 用brightkite说明

In [7]:
user_pos = pd.read_csv("Foursquare_TKY_user_xy.csv", header='infer', sep=',')
print(user_pos.columns)

Index(['# userId', 'latitude', 'longitude'], dtype='object')


In [73]:
draw_density(user_pos, save_path + "user_pos" + suffix)

In [8]:
import pickle

In [86]:
with open("Foursquare_TKY_user_wv.pkl", "rb") as f:
    wv = pickle.load(f)

In [87]:
import sklearn.cluster
import sklearn.manifold
from sklearn.metrics import silhouette_score

In [89]:
def kmeans(wv,
           n_clusters):
    '''
    Args:
        wv: A dict. dict[userId/venueId] = vec.
        n_clusters: The number of clusters.
    Returns:
        labels result.
    '''
    
    key2idx = {key: idx for idx, key in enumerate(wv.keys())}
    idx2key = {idx: key for idx, key in enumerate(wv.keys())}
    n_recs = len(wv.keys())
    
    X = np.stack([wv[idx2key[idx]] for idx in range(n_recs)], axis=0)      # [N, k]. k is the size of vec.
    kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters,
                                    random_state=21).fit(X)
    
    labels =  kmeans.labels_
    inertia = kmeans.inertia_
    sil_score = silhouette_score(X, labels)
    label_dict = {}                                                        # label_dict[userId] = label
    label_result = {label: [] for label in range(n_clusters)}              # label_result[label] = users
    for idx in range(n_recs):
        label_dict[idx2key[idx]] = labels[idx]
        label_result[labels[idx]].append(idx2key[idx])
        
    return label_dict, label_result, inertia, sil_score

In [126]:
def tsne(wv,
         n_components=2):
    '''
    Args:
        wv: A dict. dict[userId/venueId] = vec.
        n_components: The dim of the embedded space.
    Returns:
        A dict like wv. dict[userId/venueId] = embedded vec.
    '''
    
    key2idx = {key: idx for idx, key in enumerate(wv.keys())}
    idx2key = {idx: key for idx, key in enumerate(wv.keys())}
    n_recs = len(wv.keys())
    
    X = np.stack([wv[idx2key[idx]] for idx in range(n_recs)], axis=0)    # [N, k]. k is the size of vec.
    X_embedded = sklearn.manifold.TSNE(n_components=n_components).fit_transform(X) # [N, n_components].
    
    embedded_wv = {}
    for idx in range(n_recs):
        embedded_wv[idx2key[idx]] = X_embedded[idx, :]
        
    return embedded_wv

In [131]:
def spectral_clustering(wv,
                        n_clusters=8):
    '''
    Args:
        wv: A dict. dict[userId/venueId] = vec.
        n_clusters: The number of clusters.
    Returns:
        labels result.    
    '''
    
    key2idx = {key: idx for idx, key in enumerate(wv.keys())}
    idx2key = {idx: key for idx, key in enumerate(wv.keys())}
    n_recs = len(wv.keys())
    
    X = np.stack([wv[idx2key[idx]] for idx in range(n_recs)], axis=0)      # [N, k]. k is the size of vec.
    clustering = sklearn.cluster.SpectralClustering(n_clusters=n_clusters,
                                                    random_state=21,
                                                    assign_labels='discretize').fit(X)    
    labels =  clustering.labels_
    label_dict = {}                                                        # label_dict[userId] = label
    label_result = {label: [] for label in range(n_clusters)}              # label_result[label] = users
    for idx in range(n_recs):
        label_dict[idx2key[idx]] = labels[idx]
        label_result[labels[idx]].append(idx2key[idx])
        
    return label_dict, label_result

In [133]:
label_dict, label_result, inertia, sil_score = kmeans(tsne(wv), 6)

In [148]:
type(label_result[0][0])

str

In [134]:
user_pos["class"] = user_pos.apply(lambda x: label_dict[str(int(x["# userId"]))], axis=1)

In [143]:
user_pos.columns

Index(['# userId', 'latitude', 'longitude', 'class', 'class_tsne'], dtype='object')

In [163]:
fig = px.scatter_mapbox(user_pos,
                        lon='longitude',
                        lat='latitude',
                        color='class',
                        mapbox_style="open-street-map",
                        zoom=10,)
fig.write_html(save_path + "user_pos" + suffix)

In [161]:
tsne_class = 5
fig = px.density_mapbox(clean_data[clean_data["userId"].isin(label_result[tsne_class])], lat="latitude", lon="longitude", radius=1,
                        center=dict(lat=35.66, lon=139.73), zoom=10,
                        mapbox_style="open-street-map",
                        animation_frame="hour")
fig.write_html(save_path + "user_pos_clean" + str(tsne_class) + suffix)

In [ ]:
fig = px.density_mapbox(data[data["userId"]], lat="latitude", lon="longitude", radius=1,
                        center=dict(lat=35.66, lon=139.73), zoom=10,
                        mapbox_style="open-street-map",
                        animation_frame="hour")
fig.write_html(save_path + "user_pos_clean" + str(tsne_class) + suffix)

In [ ]:
fig = make_subplots(rows=3, cols=1) 

In [2]:
with open("Foursquare_TKY_user2label.pkl", "rb") as f:
    label_dict = pickle.load(f)

In [9]:
with open("Foursquare_TKY_label2user.pkl", "rb") as f:
    label_result = pickle.load(f)

In [8]:
user_pos["class"] = user_pos.apply(lambda x: label_dict[str(int(x["# userId"]))], axis=1)

In [62]:
fig = px.scatter_mapbox(user_pos,
                        lon='longitude',
                        lat='latitude',
                        color='class',
                        mapbox_style="stamen-terrain",
                        zoom=11,
                        center=dict(lat=35.66, lon=139.73))
fig.write_html(save_path + "user_pos" + suffix)

In [131]:
class_data = data[data["userId"].isin(label_result[0])]
venueCategorys = class_data["venueCategory"].to_numpy()
venueCategorys, count=np.unique(venueCategorys, return_counts=True)
idx = np.argsort(-count)
venueCategorys = venueCategorys[idx]
count = count[idx]

fig = px.bar(pd.DataFrame({"venueCategory": venueCategorys[2:20], "count": count[2:20]}),
    x="venueCategory",
    y="count",
    title="venueCategory_count")
    
fig.write_html(save_path + "VenueCategoryCount0" + suffix)

In [155]:
# fig = make_subplots(rows=2, 
#                     cols=3,
#                     subplot_titles=["类别0","类别1","类别2","类别3","类别4","类别5"]
#                    )
fig = go.Figure()
# venueCategorys = data["venueCategory"].to_numpy()
# venueCategorys, count=np.unique(venueCategorys, return_counts=True)
# idx = np.argsort(-count)
# venueCategorys = venueCategorys[idx]
# count = count[idx]
# print(venueCategorys[:30])
colors = ['#003371', '#801dae','#c93756','#f47983', '#ffa400', '#fff143']
selected_venueCategorys = ['Electronics Store', 'College Academic Building', 'Bar','Government Building','Bridge','Bus Station']
# venueCategorys = data["venueCategory"].to_numpy()
# venueCategorys=np.unique(venueCategorys)
for i in range(6):
    class_data = data[data["userId"].isin(label_result[i]) & data["venueCategory"].isin(selected_venueCategorys)]
    class_venueCategorys = class_data["venueCategory"].to_numpy()
    # print(class_venueCategorys)
    count = []
    for j in range(len(selected_venueCategorys)):
        # print(venueCategorys[2+j])
        # print(class_venueCategorys == venueCategorys[2+j])
        # count.append(sum(class_venueCategorys == venueCategorys[j])/len(data[data["venueCategory"]==venueCategorys[j]]))
        count.append(sum(class_venueCategorys == selected_venueCategorys[j])/len(class_venueCategorys))

    fig.add_trace(
        go.Bar(
        x=selected_venueCategorys,
        y=count,
        name="类" + str(i),
        marker_color=colors[i]),
    )
fig.write_html(save_path + "VenueCategoryCount2" + suffix)

In [ ]:
fig = px.histogram(data,
             x="sex",
             y="total_bill",
             color="smoker",
             barmode="stack"  # ['stack', 'group', 'overlay', 'relative']
            )

fig.show()

In [63]:
for i in range(6):
    fig = px.density_mapbox(data[data["userId"].isin(label_result[i])], lat="latitude", lon="longitude", radius=1,
                        center=dict(lat=35.66, lon=139.73), zoom=10,
                        mapbox_style="stamen-terrain",)
    fig.write_html(save_path + "userPos" + str(i)+ suffix)

In [10]:
user_data = data[data["userId"].isin(label_result[5])]
# user_data.sort_values(by="hour", inplace=True, ascending=True)
fig = px.density_mapbox(user_data, lat="latitude", lon="longitude", radius=2,
                        center=dict(lat=35.66, lon=139.73), zoom=10,
                        mapbox_style="stamen-terrain",
                        animation_frame="hour",
                        category_orders={"hour":list(range(24))})
fig.write_html(save_path + "hour5" + suffix)

In [50]:
data["hourzone"] =  data.apply(lambda x:x["hour"] // 4, axis=1)